In [3]:
# imports, add to this as needed

# change to matplotlib notebook for classic view
%matplotlib inline
from scipy.optimize import dual_annealing, basinhopping
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("darkgrid")

# Simulated Annealing for TSP with 48 cities

Adapt our "homemade" simulated annealing code from Section 2.1 in the lesson to solve the 48 capitals TSP problem from last week.  The json file `Caps48.json` in the data folder has both the distance matrix and the coordinates of the cities for plotting.  Note the distance of typical random tour is something like 80,000,000 meters and we're looking for something under 18,000,000 meters.  Your initial temperature has to be large enough to allow large moves on this scale and you'll have to increase `max_moves_no_improve` as well.  You should plot the progress of the search, like in the lesson, and you should plot the final result on the map of the United States as we did last week (that code is given in a cell below). Included a value for the random seed so that your results are reproducible.  Report a search that gives total distance < 18,000,000 meters.  A tour that achieves the optimum distance is shown in the json file.  

Put your code to find the solution in the cell below.  Your output should include a convergence plot like in the lesson (best distance versus iteration and current distance versus iteration).

<font color = "blue"> *** 4 points -  answer in cell below *** (don't delete this cell) </font>

Plot the tour on the map of the U.S.  Code to do this is below.

<font color = "blue"> *** 1 points -  answer in cell below *** (don't delete this cell) </font>

In [ ]:
# load the data and define move and objective functions
mapUS = Basemap(llcrnrlon=-119,
              llcrnrlat=22,
              urcrnrlon=-64,
              urcrnrlat=49,
              projection='lcc',
              lat_1=32,
              lat_2=45,
              lon_0=-95)

# read 48 capitals lat and lon
with open('./data/Caps48.json', 'r') as json_file:
    capitals = json.load(json_file)

xy = np.array(capitals['Coordinates'])

def plot_tour(best_tour, xy, best_dist):
    fig = plt.figure()
    fig.set_size_inches(6, 4)

    # load the shape file with "states"
    mapUS.readshapefile('./data/st99_d00', name='states', drawbounds=True)

    loop_tour = np.append(best_tour, best_tour[0])
    mapUS.plot(xy[:, 0], xy[:, 1], c='r', marker='o', markersize=4, linestyle='')
    lines, = mapUS.plot(xy[loop_tour, 0],
                      xy[loop_tour, 1],
                      c='b',
                      linewidth=1,
                      linestyle='-')
    plt.title('Best Distance {:d} km'.format(int(best_dist)))

Based on the plot of the tour do you think you have found a nearly optimal tour.  Explain why or why not.

<font color = "blue"> *** 1 points -  answer in cell below *** (don't delete this cell) </font>

# Knapsack with simanneal package

The knapsack problem is a classical combinatorial optimization problem that will be good for practicing with the ideas of discrete local search and multistart.  Given a set of items, each with a weight and a value, determine which items to include in a collection so that the total weight is less than or equal to a given limit and the total value is as large as possible.  In the 0-1 version of the knapsack problem, the decision variables are binary (or boolean) and represent whether or not to include each item in the collection.  We'll start with 20 items and you need to determine the collection of items that maximizes the value and keeps the total weight up to 50 (that is $\leq 50$).

In [9]:
# generate random weights and values for a knapsack problem
import numpy as np
num_items = 20
np.random.seed(seed=123)
values = np.random.randint(low=5, high=50, size=num_items)
weights = np.random.randint(low=1, high=10, size=num_items)
max_weight = 50
np.random.seed() # use system clock to reset the seed so future random numbers will appear random

Use the `simanneal` package to apply simulated annealing to finding a good solution to this knapsack problem. Show your solution below.  We suggest using lists of booleans to represent the items included in the knapsack as we did last week.

<font color = "blue"> *** 9 points -  answer in cell below *** (don't delete this cell) </font>

Do you think you've found the knapsack with highest possible value (the global max)?  Why or why not?

<font color = "blue"> *** 1 points -  answer in cell below *** (don't delete this cell) </font>

# 48 Capital TSP with Genetic Algorithm

Use the DEAP genetic algorithm described in the lesson to approximate a solution to the 48 state capital TSP introduced last week.  The distance matrix (in meters) and city coordinates are in `data/Caps48.json`.  Experiment with the algorithm parameters until you can find a tour of length $\leq$ 21,000,000 meters (21,000 kilometers).  Uncomment the random.seed() line and possibly try different seed values so that, if all the other parameters are the same, running the algorithm again will produce the same results.

Put your code in the cell below. Make sure it prints out both the best tour and the tour distance.  Feel free to divide distances by 1000 to display results in kilometers.

<font color = "blue"> *** 4 points -  answer in cell below *** (don't delete this cell) </font>

 Make a plot of the best tour.

<font color = "blue"> *** 1 points -  answer in cell below *** (don't delete this cell) </font>

Based on the plot of the tour do you think you have found a nearly optimal tour.  Explain why or why not.

<font color = "blue"> *** 1 points -  answer in cell below *** (don't delete this cell) </font>

# 48 Capital TSP with Genetic Algorithm and Local Search

Genetic algorithms are great for exploring a large solution space, but not so good at refining the details when close to an optimal solution.  For this reason genetic algorithms are often combined with local search.  The idea is that at each generation some or all of the individuals in the population are replaced by the result of a local search.  We'll explore this by using the 2-opt local search for TSP to refine the three worst tours in each generation.  Create a new customGA() algorithm called customGA_TSP_LS() and include this code at the beginning of the while loop:

```
# replace 3 worst individuals with local searches
pop.sort(key=lambda x:x.fitness.values,reverse=True)
num_loc_search = 3
for i in range(num_loc_search):
    best_tour, best_dist, iterations = two_opt(list(pop[i]),distance_matrix)
    pop[i] = creator.Individual(best_tour)
    pop[i].fitness.values = (best_dist,)
```
This finds the three worst tours and does a 2-opt local search on each and then replaces the results in the population.

You'll also need this version of the 2-opt search that uses the distance matrix:

In [ ]:
def sub_tour_reversal_ij(tour,i,j):
    n = len(tour)
    return (np.concatenate((tour[0:i], tour[j:-n + i - 1:-1], tour[j + 1:n])).astype(int))

def tour_distance(individual, dist_mat):
    distance = dist_mat[individual[-1]][individual[0]]
    for gene1, gene2 in zip(individual[0:-1], individual[1:]):
        distance += dist_mat[gene1][gene2]
    return (distance,) 

def two_opt(start_tour,dist_mat):
    num_cities = len(start_tour)
    current_dist = tour_distance(start_tour, dist_mat)[0]
    best_tour = start_tour
    best_dist = current_dist

    improvement = True
    iterations = 0
    while improvement:
        improvement = False
        for i in range(num_cities - 1):
            for j in range(i + 1, num_cities):
                iterations += 1
                new_tour = sub_tour_reversal_ij(best_tour, i, j)
                new_dist = tour_distance(new_tour, dist_mat)[0]
                if new_dist < best_dist:
                    best_tour = new_tour
                    best_dist = new_dist
                    improvement = True
    return best_tour, best_dist, iterations

Put your customGA_TSP_LS function in the next cell. You can run this version of the algorithm with a much smaller population and for far fewer iterations.  Find a tour of length $\leq$ 17,500 km. Use a random number seed for reproducibility.  Put your code in the next cell and print out the best distance and tour.


<font color = "blue"> *** 6 points -  answer in cell below *** (don't delete this cell) </font>

Plot the best tour below. 

<font color = "blue"> *** 1 points -  answer in cell below *** (don't delete this cell) </font>

Is this a good tour?  Explain.

<font color = "blue"> *** 1 points -  answer in cell below *** (don't delete this cell) </font>

# Knapsack problem with GA

Use the DEAP framework to build a genetic algorithm to solve the knapsack problem (same as in problem 2).  

The individuals should be lists of booleans:
```
def create_individual(n):
    return random.choices([True, False], k=n)
```

Use tournament selection and one point crossover.  For mutation flip booleans at random with this:
```
toolbox.register("mutate", tools.mutFlipBit, indpb=.1)
```

Here is a non-tuple version of a fitness function to get you started.  Notice that it includes a penalty term that penalizes if the knapsack is over the maximum weight.

```
def knapsack_value(x, values, weights, max_tot_weight):
    # x is a vector of booleans of which items to include
    tot_value = sum(values[x])
    penalty = sum(values)*min( max_tot_weight - sum(weights[x]), 0) 
    return tot_value+penalty
```
    
Put your code in the cell below.  Use random.seed() to make reproducible results.

<font color = "blue"> *** 9 points -  answer in cell below *** (don't delete this cell) </font>

Do you think you've found the knapsack with highest possible value (the global max)?  Why or why not?

<font color = "blue"> *** 1 points -  answer in cell below *** (don't delete this cell) </font>

# 30 dimensional Rastrigin Function

30 dimensions means that each individual or potential solution is a list of 30 real numbers each between -5.12 and 5.12.  Use either `simanneal` or `DEAP` (or both) to find the global optimum value (it's zero and happens when $x_1 = x_2 = \ldots = x_{30}$). Use random number seeds to make your search reproducible - `random.seed()` and/or `numpy.random.seed`, you may even need to set both since DEAP and simanneal both use `random` while we've been using `np.random` for generating moves and picking starting points for continuous optimization.   You should add a global variable to count the number of times you call the Rastrigin function and report that number as well.

<font color = "blue"> *** 8 points -  answer in cell below *** (don't delete this cell) </font>

In [6]:
# rastrigin definition
def rastrigin(x):
    x = np.array(x) # force a numpy arrray here so that the math below works
    # pass a single vector of length n (=dim) to evaluate Rastrigin
    return sum(x**2 + 10 - 10 * np.cos(2 * np.pi * x))

# add your code here:

Scipy.optimize also includes two optimization routines for continuous optimization that are similar to simulated annealing.  Both dual_annealing and basinhopping combine local search with an annealing-like process.  Run the code in the next two cells and then write about your results at the bottom.  You can try changing maxiter in dual_annealing and niter in basinhopping if you don't find the global minimum right away. 

In [ ]:
# run this cell for dual_annealing on 30D Rastrigin (mostly thanks to Deanna S.)
# first execute cell with rastrigin definition above, and imports cell at top of notebook
dim = 30
bounds = [(-5.12, 5.12) for i in range(dim)]

#this tells scipy minimize which method to use
minimizer_kwargs = {"method": "TNC"}

#define a function so we can see what's happening
log = []
acceptedVal = []
def print_fun(x, f, accepted):
        log.append(f)
        acceptedVal.append(int(accepted))

# call dual_annealing
result = dual_annealing(rastrigin, bounds, maxiter = 200, callback=print_fun)

print(
    'The lowest value of {:0.4f} took {:d} iterations and {:d} function evaluations.'
    .format(result.fun, result.nit, result.nfev))
print('The location of this lowest value is:')
print(result.x)

iterations = [i for i in range(len(log))]
import matplotlib.pyplot as plt
# plot search convergence
fig = plt.figure(figsize=(5, 3.5))
line_min, = plt.plot(iterations, log, label='Min. Val.')

plt.xlabel('Iteration')
plt.ylabel('Accepted function value')
plt.legend(handles=[line_min])
plt.title('Smallest value Found: {:4f}'.format(min(log)));

In [ ]:
# run this cell for basinhopping on 30D Rastrigin (mostly thanks to Deanna S.)
# first execute cell with rastrigin definition above, and imports cell at top of notebook

#set up the variables and initial vector
low = -5.12
high = 5.12
dim = 30
x0 = np.random.uniform(low=-5.12,high=5.12,size=dim)

#We need to use bounds, so set up a custom MyBounds object
class MyBounds(object):
    def __init__(self, xmax=[high] * dim, xmin=[low] * dim ):
        self.xmax = np.array(xmax)
        self.xmin = np.array(xmin)
    def __call__(self, **kwargs):
        x = kwargs["x_new"]
        tmax = bool(np.all(x <= self.xmax))
        tmin = bool(np.all(x >= self.xmin))
        return tmax and tmin

#create an instance of the bounds object
mybounds = MyBounds()

#this tells scipy minimize which method to use
minimizer_kwargs = {"method": "TNC"}

#define a function so we can see what's happening
log = []
acceptedVal = []
def print_fun(x, f, accepted):
        log.append(f)
        acceptedVal.append(int(accepted))

#run it
result = basinhopping(rastrigin, x0, minimizer_kwargs=minimizer_kwargs,
                   niter=200, accept_test=mybounds,
                   callback=print_fun)

print(
    'The lowest value of {:0.4f} took {:d} iterations and {:d} function evaluations.'
    .format(result.fun, result.nit, result.nfev))
print('The location of this lowest value is:')
print(result.x)

iterations = [i for i in range(len(log))]
import matplotlib.pyplot as plt
# plot search convergence
fig = plt.figure(figsize=(5, 3.5))
line_min, = plt.plot(iterations, log, label='Min. Val.')

plt.xlabel('Iteration')
plt.ylabel('Accepted function value')
plt.legend(handles=[line_min])
plt.title('Smallest value Found: {:4f}'.format(min(log)));

How does your optimization result (from DEAP and/or simanneal) compare to the results found by dual_annealing and basinhopping in terms of accuracy and efficiency?  Give a brief summary and comparision of the results.

<font color = "blue"> *** 2 points -  answer in cell below *** (don't delete this cell) </font>